In [1]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
pip install python-dotenv


Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


### Connexion à Neo4j

In [11]:
from neo4j import GraphDatabase
import os
from dotenv import load_dotenv

load_dotenv()
uri = os.getenv("NEO4J_URI")
username = os.getenv("NEO4J_USERNAME")
password = os.getenv("NEO4J_PASSWORD")
class Neo4jConnection:
    def __init__(self, uri, username, password):
        self._driver = GraphDatabase.driver(uri, auth=(username, password))

    def close(self):
        self._driver.close()

    def query(self, query, parameters=None):
        with self._driver.session() as session:
            result = session.run(query, parameters)
            return result
        

In [13]:
connection = Neo4jConnection(uri, username, password)

## chargé les fichiers csv 

In [3]:
import pandas as pd

charger le 3ème dataset 

In [ ]:
data = pd.read_csv("data/estat_isoc_eb_ics_en.csv",nrows=100)

In [98]:
industries = set(data['nace_r2'])
indicators = set(data['indic_is'])
sizes = set(data['size_emp'])
geos = set(data['geo'])


In [101]:
# Création de nœuds pour chaque entité
for industry in industries:
    query = "MERGE (:Entreprise {nace_r2: $industry})"
    connection.query(query, parameters={'industry': industry})
for indicator in indicators:
    query = "MERGE (:Indicateur {indic_is: $indicator})"
    connection.query(query, parameters={'indicator': indicator})
for size in sizes:
    query = "MERGE (:Taille {size_emp: $size})"
    connection.query(query, parameters={'size': size})
for geo in geos:
    query = "MERGE (:Geo {geo: $geo})"
    connection.query(query, parameters={'geo': geo})

# Création de relations entre les nœuds
for index, row in data.iterrows():
    query = """
    MATCH (e:Entreprise {nace_r2: $nace_r2}),
          (i:Indicateur {indic_is: $indic_is}),
          (s:Taille {size_emp: $size_emp}),
          (g:Geo {geo: $geo})
    MERGE (e)-[:HAS_INDICATOR]->(i)
    MERGE (e)-[:HAS_SIZE]->(s)
    MERGE (e)-[:LOCATED_IN]->(g)
    """
    parameters = {'nace_r2': row['nace_r2'], 'indic_is': row['indic_is'], 'size_emp': row['size_emp'], 'geo': row['geo']}
    connection.query(query, parameters)

connection.close()

C:\Users\youne\AppData\Local\Temp\ipykernel_8656\2608001907.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session() as session:


In [100]:
def delete_all_data(connection):
    query = "MATCH (n) DETACH DELETE n"
    connection.query(query)

delete_all_data(connection)

C:\Users\youne\AppData\Local\Temp\ipykernel_8656\2608001907.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session() as session:


## instanciation de la Source 2

In [102]:
data2 = pd.read_csv("data/DataCoSupplyChainDataset.csv",encoding='ISO-8859-1',nrows=100)
data2 = data2.fillna('')  

In [103]:
data2.head()

,Type,Days for shipping (real),Days for shipment (scheduled),Benefit per order,Sales per customer,Delivery Status,Late_delivery_risk,Category Id,Category Name,Customer City,...,Order Zipcode,Product Card Id,Product Category Id,Product Description,Product Image,Product Name,Product Price,Product Status,shipping date (DateOrders),Shipping Mode
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,73,Sporting Goods,Caguas,...,,1360,73,,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,2/3/2018 22:56,Standard Class
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,73,Sporting Goods,Caguas,...,,1360,73,,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/18/2018 12:27,Standard Class
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,73,Sporting Goods,San Jose,...,,1360,73,,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/17/2018 12:06,Standard Class
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,73,Sporting Goods,Los Angeles,...,,1360,73,,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/16/2018 11:45,Standard Class
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,73,Sporting Goods,Caguas,...,,1360,73,,http://images.acmesports.sports/Smart+watch,Smart watch,327.75,0,1/15/2018 11:24,Standard Class


# le schéma de l'instanciation

#### Nœud : Order
##### Propriétés :
- Order Id
- Order City
- Order Country
- Order Customer Id
- Order Date (DateOrders)
- Order Item Cardprod Id
- Order Item Discount
- Order Item Id
- Order Item Product Price
- Order Item Quantity
- Order Item Total
- Order Region
- Order State
- Order Status

#### Nœud : Shipping
##### Propriétés :
- Shipping Mode
- Days for Shipment (Scheduled)
- Days for Shipping (Real)
- Late Delivery Risk
- Delivery Status

#### Nœud : Product
##### Propriétés :
- Product Card Id
- Product Description
- Product Name
- Product Price
- Product Status
- Product Category Id
- Category Name

#### Nœud : Departement
##### Propriétés :
- Department Id
- Department Name

#### Nœud : Store
##### Propriétés :
- Latitude
- Longitude
- Department Id

#### Nœud : Customer
##### Propriétés :
- Customer Id
- Customer City
- Customer Country
- Customer Segment
- Customer Lname
- Customer State
- Customer Fname


In [104]:
data2.nunique()

Type                               4
Days for shipping (real)           7
Days for shipment (scheduled)      4
Benefit per order                 98
Sales per customer                64
Delivery Status                    4
Late_delivery_risk                 2
Category Id                        9
Category Name                      8
Customer City                     28
Customer Country                   2
Customer Email                     1
Customer Fname                    70
Customer Id                       95
Customer Lname                    70
Customer Password                  1
Customer Segment                   3
Customer State                    12
Customer Street                   94
Customer Zipcode                  30
Department Id                      5
Department Name                    5
Latitude                          95
Longitude                         56
Market                             5
Order City                        67
Order Country                     30
O

In [105]:
Customers = set(data2['Customer Id'])
Stores = set(data2['Market'])
Departments = set(data2['Department Id'])
Products = set(data2['Product Card Id'])
Shippings = set(data2['Shipping Mode'])
Orders = set(data2['Order Id'])   

In [106]:
len(Orders)

96

### Création des noeuds et propriétées

In [107]:
# Création de nœuds avec propriétés
for index, row in data2.iterrows():
    # Création du nœud Customer avec ses propriétés
    query = """
    MERGE (c:Customer {
        customer_id: $customer_id,
        city: $customer_city,
        country: $customer_country,
        segment: $customer_segment,
        lname: $customer_lname,
        state: $customer_state,
        fname: $customer_fname
    })
    """
    parameters = {
        'customer_id': row['Customer Id'],
        'customer_city': row['Customer City'],
        'customer_country': row['Customer Country'],
        'customer_segment': row['Customer Segment'],
        'customer_lname': row['Customer Lname'],
        'customer_state': row['Customer State'],
        'customer_fname': row['Customer Fname']
    }
    connection.query(query, parameters)

    # Création du nœud Store avec ses propriétés
    query = """
    MERGE (s:Store {
        market: $market,
        latitude: $latitude,
        longitude: $longitude,
        department_id: $department_id
    })
    """
    parameters = {
        'market': row['Market'],
        'latitude': row['Latitude'],
        'longitude': row['Longitude'],
        'department_id': row['Department Id']
    }
    connection.query(query, parameters)
    # Création du nœud Department avec ses propriétés
    query = """
    MERGE (d:Department {
        department_id: $department_id,
        department_name: $department_name
    })
    """
    parameters = {
        'department_id': row['Department Id'],
        'department_name': row['Department Name']
    }
    connection.query(query, parameters)

    # Création du nœud Product avec ses propriétés
    query = """
    MERGE (p:Product {
        product_card_id: $product_card_id,
        description: $product_description,
        name: $product_name,
        price: $product_price,
        status: $product_status,
        category_id: $product_category_id,
        category_name: $category_name
    })
    """
    parameters = {
        'product_card_id': row['Product Card Id'],
        'product_description': row['Product Description'],
        'product_name': row['Product Name'],
        'product_price': row['Product Price'],
        'product_status': row['Product Status'],
        'product_category_id': row['Product Category Id'],
        'category_name': row['Category Name']
    }
    connection.query(query, parameters)

    # Création du nœud Shipping avec ses propriétés
    query = """
    MERGE (sh:Shipping {
        shipping_mode: $shipping_mode,
        scheduled_days: $scheduled_days,
        real_days: $real_days,
        late_delivery_risk: $late_delivery_risk,
        delivery_status: $delivery_status
    })
    """
    parameters = {
        'shipping_mode': row['Shipping Mode'],
        'scheduled_days': row['Days for shipment (scheduled)'],
        'real_days': row['Days for shipping (real)'],
        'late_delivery_risk': row['Late_delivery_risk'],
        'delivery_status': row['Delivery Status']
    }
    connection.query(query, parameters)

    # Création du nœud Order avec ses propriétés
    query = """
    MERGE (o:Order {
        order_id: $order_id,
        city: $order_city,
        country: $order_country,
        customer_id: $order_customer_id,
        order_date: $order_date,
        item_cardprod_id: $order_item_cardprod_id,
        item_discount: $order_item_discount,
        item_id: $order_item_id,
        item_product_price: $order_item_product_price,
        item_quantity: $order_item_quantity,
        item_total: $order_item_total,
        region: $order_region,
        state: $order_state,
        status: $order_status
    })
    """
    parameters = {
        'order_id': row['Order Id'],
        'order_city': row['Order City'],
        'order_country': row['Order Country'],
        'order_customer_id': row['Order Customer Id'],
        'order_date': row['order date (DateOrders)'],
        'order_item_cardprod_id': row['Order Item Cardprod Id'],
        'order_item_discount': row['Order Item Discount'],
        'order_item_id': row['Order Item Id'],
        'order_item_product_price': row['Order Item Product Price'],
        'order_item_quantity': row['Order Item Quantity'],
        'order_item_total': row['Order Item Total'],
        'order_region': row['Order Region'],
        'order_state': row['Order State'],
        'order_status': row['Order Status']
    }
    connection.query(query, parameters)
   


C:\Users\youne\AppData\Local\Temp\ipykernel_8656\2608001907.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session() as session:


### Création des relation entre Noeuds

In [108]:
# Création de relations entre les nœuds
for index, row in data2.iterrows():
    # Relation Customer -> place_order -> Order
    query = """
    MATCH (c:Customer {customer_id: $customer_id}),
          (o:Order {order_id: $order_id})
    MERGE (c)-[:PLACE_ORDER]->(o)
    """
    parameters = {'customer_id': row['Customer Id'], 'order_id': row['Order Id']}
    connection.query(query, parameters)
    
    # Relation Order -> contains -> Product
    query = """
    MATCH (o:Order {order_id: $order_id}),
          (p:Product {product_card_id: $product_card_id})
    MERGE (o)-[:CONTAINS]->(p)
    """
    parameters = {'order_id': row['Order Id'], 'product_card_id': row['Product Card Id']}
    connection.query(query, parameters)

    # Relation Product -> belong -> Product Category
    query = """
    MATCH (p:Product {product_card_id: $product_card_id}),
          (pc:ProductCategory {category_id: $category_id})
    MERGE (p)-[:BELONG]->(pc)
    """
    parameters = {'product_card_id': row['Product Card Id'], 'category_id': row['Product Category Id']}
    connection.query(query, parameters)

    # Relation Store -> IN -> Department
    query = """
    MATCH (s:Store {market: $market}),
          (d:Department {department_id: $department_id})
    MERGE (s)-[:IN]->(d)
    """
    parameters = {'market': row['Market'], 'department_id': row['Department Id']}
    connection.query(query, parameters)

    # Relation Order -> Shipped -> Shipping
    query = """
    MATCH (o:Order {order_id: $order_id}),
          (sh:Shipping {shipping_mode: $shipping_mode})
    MERGE (o)-[:SHIPPED]->(sh)
    """
    parameters = {'order_id': row['Order Id'], 'shipping_mode': row['Shipping Mode']}
    connection.query(query, parameters)

      # Relation Store -> provide -> Product
    query = """
    MATCH (s:Store {market: $market}),
    (p:Product {product_card_id: $product_card_id})
    MERGE (s)-[:PROVIDE]->(p)
     """
    parameters = {'market': row['Market'], 'product_card_id': row['Product Card Id']}
    connection.query(query, parameters)



C:\Users\youne\AppData\Local\Temp\ipykernel_8656\2608001907.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session() as session:


In [90]:
def delete_all_data(connection):
    query = "MATCH (n) DETACH DELETE n"
    connection.query(query)

delete_all_data(connection)

C:\Users\youne\AppData\Local\Temp\ipykernel_8656\2608001907.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session() as session:


In [91]:
print(data2.columns)

Index(['Type', 'Days for shipping (real)', 'Days for shipment (scheduled)',
       'Benefit per order', 'Sales per customer', 'Delivery Status',
       'Late_delivery_risk', 'Category Id', 'Category Name', 'Customer City',
       'Customer Country', 'Customer Email', 'Customer Fname', 'Customer Id',
       'Customer Lname', 'Customer Password', 'Customer Segment',
       'Customer State', 'Customer Street', 'Customer Zipcode',
       'Department Id', 'Department Name', 'Latitude', 'Longitude', 'Market',
       'Order City', 'Order Country', 'Order Customer Id',
       'order date (DateOrders)', 'Order Id', 'Order Item Cardprod Id',
       'Order Item Discount', 'Order Item Discount Rate', 'Order Item Id',
       'Order Item Product Price', 'Order Item Profit Ratio',
       'Order Item Quantity', 'Sales', 'Order Item Total',
       'Order Profit Per Order', 'Order Region', 'Order State', 'Order Status',
       'Order Zipcode', 'Product Card Id', 'Product Category Id',
       'Product De



##### Nœud : Order
#### Propriétés :
 - Order ID
 - Order Date
 - Ship ahead day
 - Ship Late Day
 - Order Id
 - Order City
 - Order Country
 - Order Customer Id
 - order date (DateOrders)
 - Order Item Cardprod Id
 - Order Item Discount
 - Order Item Id
 - Order Item Product Price
 - Order Item Quantity
 - Order Item Total
 - Order Region
 - Order State
 - Order Status
##### Nœud : Product
#### Propriétés :
 - Product ID
 - Cost per Unit
 - Unit quantity
 - Product Card Id
 - Product Description
 - Product Name
 - Product Price
 - Product Status
 - Product Category Id
 - Category Name
##### Nœud : Customer
#### Propriétés :
 - Customer 
 - Customer Id
 - Customer City
 - Customer Country
 - Customer Segment
 - Customer Lname
 - Customer State
 - Customer Fname
##### Nœud : Carrier
#### Propriétés :
 - Carrier 
 - Carrier Mode
 - TPT
##### Nœud : Port
#### Propriétés :
 - Port 
 - Destination Port
 - Origin Port
##### Nœud : Usine
#### Propriétés :
 - Plant Code
 - Daily Capacity
##### Nœud : Shipping
#### Propriétés :
 - Shipping Mode
 - Days for shipment (scheduled)
 - Days for shipping (real)
 - Late_delivery_risk
 - Delivery Status
#### Nœud : Departement
##### Propriétés :
 - Department Id
 - Department Name
#### Nœud : Store
##### Propriétés :
 - Latitude
 - Longitude
 - Department Id
#### Nœud : Entreprise
##### Propriétés :
 - indic_is
 - nace_r2
#### Nœud : Size
##### Propriétés :
 - size_emp
#### Nœud : Geo
##### Propriétés :
 - geo

## instanciation manuelle des 3 sources

In [23]:
data3 = pd.read_excel("data/Supply chain logisitcs problem.xlsx",nrows=200)

In [93]:
data3.head()

,Order ID,Order Date,Origin Port,Carrier,TPT,Service Level,Ship ahead day count,Ship Late Day count,Customer,Product ID,Plant Code,Destination Port,Unit quantity,Weight
0,1.447296e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,808,14.30
1,1.447158e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,3188,87.94
2,1.447139e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2331,61.20
3,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,847,16.16
4,1.447364e+09,2013-05-26,PORT09,V44_3,1,CRF,3,0,V55555_53,1700106,PLANT16,PORT09,2163,52.34


In [7]:
data3.nunique()

Order ID                9215
Order Date                 1
Origin Port                3
Carrier                    3
TPT                        5
Service Level              3
Ship ahead day count       5
Ship Late Day count        4
Customer                  46
Product ID               772
Plant Code                 7
Destination Port           1
Unit quantity           2624
Weight                  5899
dtype: int64

In [8]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9215 entries, 0 to 9214
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Order ID              9215 non-null   float64       
 1   Order Date            9215 non-null   datetime64[ns]
 2   Origin Port           9215 non-null   object        
 3   Carrier               9215 non-null   object        
 4   TPT                   9215 non-null   int64         
 5   Service Level         9215 non-null   object        
 6   Ship ahead day count  9215 non-null   int64         
 7   Ship Late Day count   9215 non-null   int64         
 8   Customer              9215 non-null   object        
 9   Product ID            9215 non-null   int64         
 10  Plant Code            9215 non-null   object        
 11  Destination Port      9215 non-null   object        
 12  Unit quantity         9215 non-null   int64         
 13  Weight            

In [9]:
plantPort = pd.read_excel("data/Supply chain logisitcs problem.xlsx", sheet_name=6) 

In [10]:
plantPort

,Plant Code,Port
0,PLANT01,PORT01
1,PLANT01,PORT02
2,PLANT02,PORT03
3,PLANT03,PORT04
4,PLANT04,PORT05
5,PLANT05,PORT06
6,PLANT06,PORT06
7,PLANT07,PORT01
8,PLANT07,PORT02
9,PLANT08,PORT04


### création du Noeud Port à partir de la feuille 7

In [15]:
for index, row in plantPort.iterrows():
    query = """
    MERGE (p:Port {
    port_id: $port_id
    })
    """
    parameters = {
        'port_id': row['Port']
    }
    connection.query(query,parameters)

    

In [21]:
# Création des nœuds partagés (Product, Order, Customer)
for index, row in data3.iterrows():
    # Création du nœud Product avec ses propriétés
    query = """
    MERGE (p:Product {
        product_id: $product_id,
        unit_quantity: $unit_quantity
    })
    """
    parameters = {
        'product_id': row['Product ID'],
        'unit_quantity': row['Unit quantity']
    }
    connection.query(query, parameters)

    # Création du nœud Order avec ses propriétés
    query = """
    MERGE (o:Order {
        order_id: $order_id,
        order_date: $order_date,
        ship_ahead_day: $ship_ahead_day,
        ship_late_day: $ship_late_day
    })
    """
    parameters = {
        'order_id': row['Order ID'],
        'order_date': row['Order Date'],
        'ship_ahead_day': row['Ship ahead day count'],
        'ship_late_day': row['Ship Late Day count']
    }
    connection.query(query, parameters)

    # Création du nœud Customer avec ses propriétés
    query = """
    MERGE (c:Customer {
        customer: $customer
    })
    """
    parameters = {'customer': row['Customer']}
    connection.query(query, parameters)

    # Création des nœuds spécifiques à la deuxième source de données (Carrier, Port, Usine)
    query = """
    MERGE (ca:Carrier {
        carrier: $carrier,
        tpt: $tpt
    })
    """
    parameters = {
        'carrier': row['Carrier'],
        'tpt': row['TPT']
    }
    connection.query(query, parameters)

    query = """
    MERGE (u:Usine {
        plant_code: $plant_code
    })
    """
    parameters = {
        'plant_code': row['Plant Code']
    }
    connection.query(query, parameters)

# Gestion des relations entre les nœuds
# Vous devez ajouter la logique pour créer les relations entre les nœuds selon vos besoins.


## creation des relation du nouveau graph

In [112]:
data3.columns

Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')

In [117]:
# Création de relations entre les nœuds
for index, row in data3.iterrows():
    # Relation Customer -> place_order -> Order
    query = """
    MATCH (c:Customer {customer_id: $customer_id}),
          (o:Order {order_id: $order_id})
    MERGE (c)-[:PLACE_ORDER]->(o)
    """
    parameters = {'customer_id': row['Customer'], 'order_id': row['Order ID']}
    connection.query(query, parameters)
    
    # Relation Order -> contains -> Product
    query = """
    MATCH (o:Order {order_id: $order_id}),
          (p:Product {product_id: $product_id})
    MERGE (o)-[:CONTAINS]->(p)
    """
    parameters = {'order_id': row['Order ID'], 'product_id': row['Product ID']}
    connection.query(query, parameters)

    # Relation Usine -> provide -> Product
    query = """
    MATCH (u:Usine {plant_code: $plant_code}),
          (p:Product {product_id: $product_id})
    MERGE (u)-[:PROVIDE]->(p)
    """
    parameters = {'plant_code': row['Plant Code'], 'product_id': row['Product ID']}
    connection.query(query, parameters)

  
    
    # Relation Carrier -> transport -> Order
    query = """
    MATCH (ca:Carrier {carrier: $carrier}),
          (o:Order {order_id: $order_id})
    MERGE (ca)-[:TRANSPORT]->(o)
    """
    parameters = {'carrier': row['Carrier'], 'order_id': row['Order ID']}
    connection.query(query, parameters)


C:\Users\youne\AppData\Local\Temp\ipykernel_8656\2608001907.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with self._driver.session() as session:


### création de la relation entre port et Usine

In [22]:
  # Relation Usine -> Based -> Port
for index, row in plantPort.iterrows():
    query = """
    MATCH (u:Usine {plant_code: $plant_code}),
          (po:Port {port_id: $port})
    MERGE (u)-[:BASED]->(po)
    """
    parameters = {'plant_code': row['Plant Code'], 'port': row['Port']}
    connection.query(query, parameters)

    query = """
    MERGE (u:Usine {
        plant_code: $plant_code
    })
    """
    parameters = {
        'plant_code': row['Plant Code']
    }
    connection.query(query, parameters)